In [13]:
import vmodel
import os
import numpy as np
import h5py
import datetime
import scipy.spatial
import math


class Vect:

   def __init__(self, a, b):
        self.a = a
        self.b = b

   def findClockwiseAngle(self, other):
       # using cross-product formula
       return -math.degrees(math.asin((self.a * other.b - self.b * other.a)/(self.length()*other.length())))
       # the dot-product formula, left here just for comparison (does not return angles in the desired range)
       # return math.degrees(math.acos((self.a * other.a + self.b * other.b)/(self.length()*other.length())))

   def length(self):
       return math.sqrt(self.a**2 + self.b**2)

def calc_order(vel: np.ndarray) -> float:
    """Compute order parameter from velocity matrix
    Args:
        vel: velocity matrix (N x D)
    Returns:
        order: velocity correlation
    """
    N, _ = vel.shape
    speed = np.linalg.norm(vel, axis=1, keepdims=True)  # N x 1
    speed_prod = speed.dot(speed.T)  # N x N
    mask = (speed_prod != 0)  # avoid division by zero!
    dot_prod = vel.dot(vel.T)  # N x N
    np.fill_diagonal(dot_prod, 0)  # i != j
    return (dot_prod[mask] / speed_prod[mask]).sum() / (N * (N - 1))
out_str = "/home/lars/vmodel_output/relax_100_Nocol_occ/"
saveLoc = "/home/lars/vmodel_output/"
saveName = "test_relax100_NoCol_occ"


args = {
'nprey': 100,
'npred': 1,
'frange': 10,
'fstr': 50.0,
'visPred': 300.0,
'visPrey': 330,
'astr': 3,
'dphi': 0.2,
'repPrey': 3,
'repRadPrey': 1.5,
'repPred': 21,  
'repRadPred': 20,
'attPrey': 3,
'attRadPrey': 1.5,
'repCol': 10000000,
'hstr': 1,
'steps': 4000,
    }

paraChange1_name = "fangle"
paraChange2_name = "pangle"
steps = 20 #10
reps = 1 #20
pred_time = 1200

total = steps*steps*reps

paraChange1_val = np.linspace(0,3,steps)
paraChange2_val = np.linspace(0,1,steps)

paraChange1_val = [30.0]
paraChange2_val = [0] #,90,180]


mindist_hm = []

time_now = datetime.datetime.now()
time_elapsed = 0
NN_scan = np.zeros((steps, steps, reps, args["steps"]-pred_time))



for i in range(len(paraChange1_val)):
    
    for j in range(len(paraChange2_val)):

        #np.savetxt(str(saveLoc)+""+str(saveName)+"pol_"+str(paraChange1_name)+"_"+str(paraChange2_name)+".csv", pol_scan, delimiter=",")
  
        #pol_reps = np.zeros((reps, args["steps"]))
        IID_reps = []
        CND_reps = []
        
        args[paraChange1_name] = paraChange1_val[i]
        args[paraChange2_name] = paraChange2_val[j]

        npred = args["npred"]
        nprey = args["nprey"]
        

        args_str = '_'.join(f'{k}_{v}' for k, v in args.items())

        file_h5 = f'{out_str}_{args_str}.states.nc'
        file_h5 = "/home/lars/vmodel_output/_nprey_100_npred_1_frange_10_fstr_0.0_visPred_300.0_visPrey_330_astr_3_dphi_0.2_repPrey_3_repRadPrey_1.5_repPred_21_repRadPred_20_attPrey_3_attRadPrey_1.5_repCol_10000000_hstr_1_steps_4000_fangle_30.0_pangle_0.states.nc"

        print(file_h5)
        


        try:
            with h5py.File(file_h5) as fh5:
                #vel = np.moveaxis(np.array(fh5['/velocity']), [3,2], [1,3])[:,:,:,:]
                #pos = np.moveaxis(np.array(fh5['/position']), [3,2], [1,3])[:,:,:,:]
                vis = np.array(fh5['/visibility'])[:,:100,:100,:]

        except:
            print("File not Found, going on")
        
        print(np.shape(vis))

        for rep in range(reps):
            
            print(rep)
            
            #vel_rep = vel[rep,:,:nprey,:]
            #pos_rep = pos[rep,:,:nprey,:]
            vis_rep = vis[rep,:,:,:]


            time, N, dim = np.shape(vis_rep)
            NN_scan[i, j, rep] = np.mean(vis_rep)
            print(np.mean(vis_rep))

/home/lars/vmodel_output/_nprey_100_npred_1_frange_10_fstr_0.0_visPred_300.0_visPrey_330_astr_3_dphi_0.2_repPrey_3_repRadPrey_1.5_repPred_21_repRadPred_20_attPrey_3_attRadPrey_1.5_repCol_10000000_hstr_1_steps_4000_fangle_30.0_pangle_0.states.nc
(1, 100, 100, 2800)
0
0.051505214285714286


In [11]:
np.shape(vis_rep)

(100, 100, 2800)

In [15]:
np.sum(vis_rep[0,:,2000])

5

In [18]:
np.mean(np.sum(vis_rep, axis = 1))

5.1505214285714285